In [ ]:
#Run the setup script
!chmod +x setup.sh
!./setup.sh

In [9]:
#Run the demo
!python code_generation.py --prompts_file_name "openai_humaneval.jsonl" --model_name "Salesforce/codegen-350M-mono" --generation_strategy "ammarnasr/pass_at_1_gen_config" --output_file_name "codegen350Mmono_pass1_humaneval_python.jsonl" --device "cuda" 

^C
